In [13]:
import numpy as np
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import time
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer


In [14]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())
df = df[3].tolist()
df = df[:1000]

In [15]:
len(df)

1000

In [16]:
len(set(df))

506

In [17]:
df = np.array(df, dtype='object')


In [18]:
df

array(['488846200', '508455704', '508455712', '508455720', '508500288',
       '508500296', '508500304', '508516672', '508516680', '508516688',
       '283193112', '283193120', '283193120', '283193128', '283193128',
       '283193136', '283193136', '283193144', '283193144', '283193152',
       '283193152', '283193160', '283193160', '283193168', '283193168',
       '283193176', '283193176', '283193184', '283193184', '283193208',
       '283193208', '283193216', '283193216', '283193224', '283193224',
       '283193232', '283193232', '283193240', '283193240', '283193248',
       '283193248', '283193256', '283193256', '283193264', '283193264',
       '283193272', '283193272', '283193280', '283193280', '283193288',
       '283193288', '283193296', '283193296', '283193304', '283193304',
       '283193192', '283193192', '283193200', '283193200', '283193312',
       '283193312', '283193320', '283193320', '283193328', '283193328',
       '283193336', '283193336', '283193344', '283193344', '2831

In [22]:
'''
get the furthest accessed block. Scans OPT dictionary and selects the maximum positioned element
'''

def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            #print ( "Not Acccessing block anymore " + str(cached_block))
            return cached_block            
    for cached_block in C:  
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    #print ( "chose to evict " + str(maxAccessBlock) + " since its position of access is " + str(maxAccessPosition))
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    OPT = defaultdict(partial(np.ndarray,0))

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block] = np.append(OPT[block], i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = set()
    d = {}
    seq_number = 0
    for block in tqdm(blocktrace, desc="OPT", leave=False):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block] = np.delete(OPT[block],0)
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                d[fblock] = C
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block] = np.delete(OPT[block],[0])

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    #hitrate = hit / (hit + miss)
    #print(hitrate)
    return d

In [23]:
d = belady_opt(df, 10)

In [24]:
for i,j in d.items():
    d[i] = list(map(str, j))

In [25]:
data = []
label = []
for i,j in tqdm(d.items()):
    data.append(' '.join(j))
    label.append(i)

In [26]:
data

['283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 283193360 283195712 283194088 508516680',
 '283197064 283193128 283193200 283196584 283196928 283194416 28

In [28]:
len(set(label))

496

In [29]:
vec = CountVectorizer(binary=True)
vec_ = vec.fit(df)
DATA = vec_.transform(data)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(DATA, label ,test_size=0.5, shuffle=True)

In [31]:
clf = LogisticRegression(dual=True).fit(X_train, Y_train)

In [32]:
accuracy_score(Y_test, clf.predict(X_test))

0.0

In [80]:
X_train.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [81]:
clf.predict(vec_.transform())

array(['488846200'], dtype='<U9')

In [25]:
max(clf.predict_proba(X_test)[0])

0.06592931565563544

In [30]:
st = clf.predict_proba(X_test)[0]

In [31]:
s = np.where(st == max(st))

In [32]:
s

(array([109]),)

In [38]:
Y_test[109]

'283195152'